### Q1. Notebook

In [7]:
!pip freeze | grep scikit-learn

scikit-learn==1.5.0


In [8]:
!python -V

Python 3.10.13


In [9]:
import pickle
import pandas as pd
import os

In [10]:
with open('model.bin', 'rb') as f_in:
    dv, model = pickle.load(f_in)

In [11]:
# Option 1: Install pyarrow
!pip install pyarrow

# Option 2: Install fastparquet
!pip install fastparquet

In [12]:
categorical = ['PULocationID', 'DOLocationID']

def read_data(filename):
    df = pd.read_parquet(filename)
    
    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df['duration'] = df.duration.dt.total_seconds() / 60

    df = df[(df.duration >= 1) & (df.duration <= 60)].copy()

    df[categorical] = df[categorical].fillna(-1).astype('int').astype('str')
    
    return df

In [13]:
df = read_data('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-03.parquet')

In [14]:
dicts = df[categorical].to_dict(orient='records')
X_val = dv.transform(dicts)
y_pred = model.predict(X_val)

In [15]:
(y_pred).std()

6.247488852238703

### Q2. Preparing the output

In [16]:
year = 2023
month = 3

In [17]:
df['ride_id'] = f'{year:04d}/{month:02d}_' + df.index.astype('str')

In [18]:
df_result = pd.DataFrame()
df_result['ride_id'] = df['ride_id']
df_result['Prediction'] = y_pred

In [21]:
df_result.to_parquet(
    'output.parquet',
    engine='pyarrow',
    compression=None,
    index=False
)

In [22]:
file_stats = os.stat('output.parquet')
file_stats.st_size

68641760